In [ ]:
import requests
from IPython.core.display import HTML
import pandas as pd
from bs4 import BeautifulSoup as BS
import re


In [ ]:
#Assigning URLs of each dog treat results page
url  = "https://www.petsmart.com/dog/treats/biscuits-and-bakery/?pmin=0.01&srule=best-sellers&start=0sz=36"
url2 = "https://www.petsmart.com/dog/treats/biscuits-and-bakery/?pmin=0.01&srule=best-sellers&start=36&sz=36"
url3 = "https://www.petsmart.com/dog/treats/biscuits-and-bakery/?pmin=0.01&srule=best-sellers&start=72&sz=36"
url4 = "https://www.petsmart.com/dog/treats/biscuits-and-bakery/?pmin=0.01&srule=best-sellers&start=108&sz=36"
url5 = "https://www.petsmart.com/dog/treats/biscuits-and-bakery/?pmin=0.01&srule=best-sellers&start=144&sz=36"

In [ ]:
data = requests.get(url)
data2 = requests.get(url2)
data3 = requests.get(url3)
data4 = requests.get(url4)
data5 = requests.get(url5)

In [ ]:
data.status_code

In [ ]:
#Creating soup of each response
soup  = BS(data.text)
soup2 = BS(data2.text)
soup3 = BS(data3.text)
soup4 = BS(data4.text)
soup5 = BS(data5.text)

In [ ]:
#finding the necessary links to a product page.
soup2.find_all('a', {'class':'name-link'}, href=True)

In [ ]:
#creating empty lists to store product links in
links  = []
links2 = []
links3 = []
links4 = []
links5 = []

In [ ]:
#for loops extracting links from each page
for a in soup.find_all('a', {'class':'name-link'}, href=True):
    links.append(a['href'])

In [ ]:
for a in soup2.find_all('a', {'class':'name-link'}, href=True):
    links2.append(a['href'])

In [ ]:
for a in soup3.find_all('a', {'class':'name-link'}, href=True):
    links3.append(a['href'])

In [ ]:
for a in soup4.find_all('a', {'class':'name-link'}, href=True):
    links4.append(a['href'])

In [ ]:
for a in soup5.find_all('a', {'class':'name-link'}, href=True):
    links5.append(a['href'])

In [ ]:
#combining lists together
alllinks = links + links2 + links3 + links4 + links5

In [ ]:
#creating dataframe object of links
treatsdf = pd.DataFrame(alllinks)

In [ ]:
#renaming column
treatsdf.columns = ['link']

In [ ]:
treatsdf.head()

In [ ]:
#completing the url for each extracted link
treatsdf['urls'] = 'https://www.petsmart.com' + treatsdf['link']

In [ ]:
#simplifying dataframe
treatsdf = treatsdf[['urls']]

In [ ]:
treatsdf.head()

#   

In [ ]:
#importing selenium to read information from a given products page
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [ ]:
#establishing browser for Selenium to use.
browser = webdriver.Chrome()

In [ ]:
testurl = 'https://www.petsmart.com/dog/treats/biscuits-and-bakery/kong-ziggiesandtrade-dog-treat-9903.html?cgid=100266&fmethod=Browse'

In [ ]:
browser.get(testurl)

In [ ]:
#extracting product name from testurl
browser.find_element(By.CLASS_NAME, 'pdp-product-name').text

In [ ]:
#extracting product price from testurl
browser.find_element(By.CLASS_NAME, 'product-price-standard').text.strip()

In [ ]:
#extracting rating from testurl
browser.find_element(By.CLASS_NAME, 'bv-off-screen').text

In [ ]:
#creating a click to extract necessary element
browser.find_element(By.XPATH, '/html/body/div[3]/div[1]/div[2]/div[5]/div/div/div[6]/div/div/div/div/div/ul/li[2]/div').click()

In [ ]:
#extracting ingredients from testurl
browser.find_element(By.XPATH, '/html/body/div[3]/div[1]/div[2]/div[5]/div/div/div[6]/div/div/div/div/div/div[2]/div/p[1]').text.strip()

In [ ]:
#creating empty lists for additional information
product = []
price = []
size = []
rating = []
ingredients = []

for url in treatsdf['urls']:
    browser.get(url)
    
    #Sleeping to allow the page to fully load all Java elements.
    time.sleep(15)
    
    #Getting product name
    try:
        product.append(browser.find_element(By.CLASS_NAME, 'pdp-product-name').text)
    except:
        product.append('n/a')
    
    #Getting product price
    try:
        price.append(browser.find_element(By.CLASS_NAME, 'product-price-sales').text)
    except:
        try:
            price.append(browser.find_element(By.CLASS_NAME, 'product-price-standard').text)
        except:
            price.append('n/a')
            
    #Getting product rating
    try:
        rating.append(browser.find_element(By.CLASS_NAME, 'bv-off-screen').text)
    except:
        rating.append('n/a')
    
    #Getting product size
    try:
        size.append(browser.find_element(By.ID, 'size').text)
    except:
        try:
            size.append(browser.find_element(By.ID, 'customQuantity').text)
        except:
            size.append('n/a')
            
    #Getting product ingredients
    try:
        browser.find_element(By.XPATH, '/html/body/div[3]/div[1]/div[2]/div[5]/div/div/div[6]/div/div/div/div/div/ul/li[2]/div').click()
        ingredients.append(browser.find_element(By.XPATH, '/html/body/div[3]/div[1]/div[2]/div[5]/div/div/div[6]/div/div/div/div/div/div[2]/div/p[1]').text)
    except:
        try:
            ingredients.append(browser.find_element(By.XPATH, '/html/body/div[3]/div[1]/div[2]/div[5]/div/div/div[6]/div/div/div/div/div/div/div/p[3]').text)
        except:
            try:
                browser.find_element(By.XPATH, '/html/body/div[3]/div[1]/div[2]/div[5]/div/div/div[5]/div/div/div/div/div/ul/li[2]/div').click()
                ingredients.append(browser.find_element(By.XPATH, '/html/body/div[3]/div[1]/div[2]/div[5]/div/div/div[5]/div/div/div/div/div/div[2]/div/p[1]').text)
            except:
                ingredients.append('n/a')

In [ ]:
product

In [ ]:
size

In [ ]:
price

In [ ]:
ingredients

In [ ]:
rating

In [ ]:
len(alllinks)

In [ ]:
treatsdf['product'] = product
treatsdf['price'] = price
treatsdf['size'] = size
treatsdf['rating'] = rating
treatsdf['ingredients'] = ingredients

In [ ]:
#treatsdf.to_csv('../data/treatsdf.csv', index=False)

In [ ]:
treatsdf.head()

In [ ]:
driver.quit()